***Web Scrapping***

**Scrapper une page des avis sur un film**

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import pandas as pd

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115.0 Safari/537.36"
}
URL = "https://www.allocine.fr/film/fichefilm_gen_cfilm=143692.html"  # Exemple: Film Inception


def get_soup(url):
    """Récupère le contenu HTML d'une page et le parse avec BeautifulSoup"""
    r = requests.get(url, headers=HEADERS, timeout=10)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")


def find_spectator_page(soup, base_url):
    """Cherche le lien vers la page 'critiques spectateurs'"""
    for a in soup.find_all("a", href=True):
        txt = (a.get_text() or "").strip().lower()
        href = a["href"]
        if "spectateur" in txt or "spectateurs" in txt or "critique spectateur" in txt:
            return urljoin(base_url, href)
        if "critiques" in href and "spectateur" in href:
            return urljoin(base_url, href)
    return None


def parse_note_from_element(el):
    """Essaie d'extraire la note d'un bloc de critique"""
    sel = el.select_one("span.stareval-note, span[class*='stareval'], span[class*='note'], span[class*='rating']")
    if sel and sel.get_text(strip=True):
        s = re.sub(r"[^\d,\.]", "", sel.get_text(strip=True)).replace(",", ".")
        try:
            return float(s)
        except:
            pass
    for attr in ("data-note", "data-rating", "data-score", "data-rate", "data-rate-value"):
        if el.has_attr(attr):
            s = re.sub(r"[^\d\.]", "", str(el[attr]).replace(",", "."))
            try:
                return float(s)
            except:
                pass
    for img in el.find_all("img"):
        alt = img.get("alt", "")
        m = re.search(r"(\d+(?:[.,]\d+)?)", alt)
        if m:
            try:
                return float(m.group(1).replace(",", "."))
            except:
                pass
    txt = el.get_text(" ", strip=True)
    m = re.search(r"(\d(?:[.,]\d)?)\s*(?:/|sur)\s*5", txt)
    if m:
        try:
            return float(m.group(1).replace(",", "."))
        except:
            pass
    return None


def extract_reviews_from_soup(soup, base_url):
    """Récupère les avis utilisateurs et leurs notes"""
    reviews = []
    review_blocks = soup.select("div.review-card-content, p.content-txt")

    for rb in review_blocks[:200]:
        review_text = rb.get_text(" ", strip=True)
        if review_text:
            note = None

            # Chercher la note dans le frère précédent (ligne juste avant le commentaire)
            prev_sib = rb.find_previous_sibling()
            if prev_sib:
                note = parse_note_from_element(prev_sib)

            # fallback : chercher dans le parent si pas trouvé
            if note is None and rb.parent:
                note = parse_note_from_element(rb.parent)

            reviews.append({
                "review": review_text,
                "user_note": note
            })

    return reviews


def scrape_allocine(url):
    """Scrape le titre du film et les avis utilisateurs"""
    soup_home = get_soup(url)
    spec_url = find_spectator_page(soup_home, url)
    soup = get_soup(spec_url) if spec_url else soup_home

    title_tag = soup_home.find("h1")
    title = title_tag.get_text(strip=True) if title_tag else "Titre inconnu"

    reviews = extract_reviews_from_soup(soup, url)
    return title, reviews


if __name__ == "__main__":
    title, reviews = scrape_allocine(URL)
    print("Film :", title)
    print("Nombre d'avis trouvés (approx.):", len(reviews))
    for i, r in enumerate(reviews[:10], 1):
        print(f"\nAvis #{i}: note = {r['user_note']}  ---  {r['review'][:160]}{'...' if len(r['review'])>160 else ''}")

    df = pd.DataFrame(reviews)
    df["film_title"] = title
    df["film_url"] = URL
    df.to_csv("avis_films_user_comments.csv", index=False, encoding="utf-8")
    print("\nCSV sauvegardé: avis_films_user_comments.csv")


Film : Inception de Christopher Nolan
Nombre d'avis trouvés (approx.): 15

Avis #1: note = 5.0  ---  Après le chef d'oeuvre super-héroïque The Dark Knight, Christopher Nolan a carte blanche pour réaliser son prochain long-métrage. Il se met alors à la réalisati...

Avis #2: note = 5.0  ---  C’est fou ce qu’on aime détester Christopher Nolan… Plus ses films sortent et plus ça jase sur lui, sur ce qu’il fait, sur ce qu’il prétend faire… Et là vu qu’a...

Avis #3: note = 5.0  ---  Un film aussi novateur que complexe, dont la mise en scène touche la perfection, nous congratulant de scènes d'action mémorables et d'un casting resplendissant....

Avis #4: note = 5.0  ---  CHEF D’ŒUVRE ! Le film est absolument parfait ! les acteurs sont tous excellent, le scénario est génial et surtout très originale, les effets spéciaux parfaits,...

Avis #5: note = 5.0  ---  Nolan est un vrai génie. On l'avait déjà entraperçu dans ses premiers films, comme " Le Prestige " ou " The Dark Knight " et ça se conf

**Scrapper des avis sur plusieurs films**

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import pandas as pd

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115.0 Safari/537.36"
}

#Tâche 1 : Automatisation de la collecte
URLS = [
    "https://www.allocine.fr/film/fichefilm_gen_cfilm=2954.html",  # Le Prête-nom
    "https://www.allocine.fr/film/fichefilm_gen_cfilm=20297.html",   # Séduite et abandonnée
    "https://www.allocine.fr/film/fichefilm_gen_cfilm=61282.html",   # Avatar
    "https://www.allocine.fr/film/fichefilm_gen_cfilm=11010.html",   # Box of Moonlight
    "https://www.allocine.fr/film/fichefilm_gen_cfilm=23170.html",   # Le Combat de Kyoshiro Nemuri
    "https://www.allocine.fr/film/fichefilm_gen_cfilm=14637.html",   # Marie et le curé
    "https://www.allocine.fr/film/fichefilm_gen_cfilm=32745.html",   # Youngblood
    "https://www.allocine.fr/film/fichefilm_gen_cfilm=8292.html",   # L'Avocat du diable
    "https://www.allocine.fr/film/fichefilm_gen_cfilm=32741.html" # Fanfan
]

def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=10)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

def find_spectator_page(soup, base_url):
    for a in soup.find_all("a", href=True):
        txt = (a.get_text() or "").strip().lower()
        href = a["href"]
        if "spectateur" in txt or "spectateurs" in txt or "critique spectateur" in txt:
            return urljoin(base_url, href)
        if "critiques" in href and "spectateur" in href:
            return urljoin(base_url, href)
    return None

def parse_note_from_element(el):
    sel = el.select_one("span.stareval-note, span[class*='stareval'], span[class*='note'], span[class*='rating']")
    if sel and sel.get_text(strip=True):
        s = re.sub(r"[^\d,\.]", "", sel.get_text(strip=True)).replace(",", ".")
        try:
            return float(s)
        except:
            pass
    for attr in ("data-note", "data-rating", "data-score", "data-rate", "data-rate-value"):
        if el.has_attr(attr):
            s = re.sub(r"[^\d\.]", "", str(el[attr]).replace(",", "."))
            try:
                return float(s)
            except:
                pass
    for img in el.find_all("img"):
        alt = img.get("alt", "")
        m = re.search(r"(\d+(?:[.,]\d+)?)", alt)
        if m:
            try:
                return float(m.group(1).replace(",", "."))
            except:
                pass
    txt = el.get_text(" ", strip=True)
    m = re.search(r"(\d(?:[.,]\d)?)\s*(?:/|sur)\s*5", txt)
    if m:
        try:
            return float(m.group(1).replace(",", "."))
        except:
            pass
    return None

def extract_reviews_from_soup(soup, base_url):
    reviews = []
    review_blocks = soup.select("div.review-card-content, p.content-txt")

    for rb in review_blocks[:200]:
        review_text = rb.get_text(" ", strip=True)
        # Supprimer la première ligne si c'est le pseudo ou infos utilisateur
        lines = review_text.splitlines()
        if len(lines) > 1:
            review_text = " ".join(lines[1:]).strip()

        if review_text:
            note = None
            prev_sib = rb.find_previous_sibling()
            if prev_sib:
                note = parse_note_from_element(prev_sib)
            if note is None and rb.parent:
                note = parse_note_from_element(rb.parent)
            reviews.append({
                "review": review_text,
                "user_note": note
            })
    return reviews

def scrape_allocine(url):
    soup_home = get_soup(url)
    spec_url = find_spectator_page(soup_home, url)
    soup = get_soup(spec_url) if spec_url else soup_home

    title_tag = soup_home.find("h1")
    title = title_tag.get_text(strip=True) if title_tag else "Titre inconnu"

    reviews = extract_reviews_from_soup(soup, url)
    return title, reviews

#Tâche 2 : Collecter les données pour tous les films
all_data = []
for url in URLS:
    try:
        title, reviews = scrape_allocine(url)
        for r in reviews:
            all_data.append({
                "film_title": title,
                "film_url": url,
                "user_note": r["user_note"],
                "review": r["review"]
            })
        print(f"[INFO] {len(reviews)} avis récupérés pour {title}")
    except Exception as e:
        print(f"[ERROR] Impossible de scraper {url}: {e}")

#Tâche 3 : Créer DataFrame et sauvegarder CSV
df = pd.DataFrame(all_data)
df.to_csv("avis_films_bruts.csv", index=False, encoding="utf-8")
print("\n Dataset sauvegardé dans avis_films_bruts.csv")


[INFO] 11 avis récupérés pour Le Prête-nom de Martin Ritt
[INFO] 14 avis récupérés pour Séduite et abandonnée de Pietro Germi
[INFO] 15 avis récupérés pour Avatar de James Cameron
[INFO] 14 avis récupérés pour Box of Moonlight de Tom DiCillo
[INFO] 2 avis récupérés pour Le Combat de Kyoshiro Nemuri de Kenji Misumi
[INFO] 3 avis récupérés pour Marie et le curé court-métrage de Diourka Medvedzki
[INFO] 15 avis récupérés pour Youngblood de Peter Markle
[INFO] 15 avis récupérés pour L'Avocat du diable de Sidney Lumet
[INFO] 15 avis récupérés pour Fanfan de Alexandre Jardin

 Dataset sauvegardé dans avis_films_bruts.csv
